In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS
import pathlib
from pathlib import Path
from shapely import wkt
from tqdm import tqdm

import math
import codecs
import osm2geojson
from shapely import wkt

import gzip
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import xml.etree.ElementTree as ET

import seaborn as sns

# set the working directory
BASE_DIR = Path.cwd()

# auto reloading
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# read the SWITRS raw records. 
dfSWITRS = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","CollisionRecords.txt"))
dfSWITRS.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Raw_SWITRS.csv"))
# The file downloaded from the SWITRS website has records from 2010 to 2016.So keep only those year records
dfSWITRS=dfSWITRS[dfSWITRS["ACCIDENT_YEAR"].isin([2010,2016])]
"""
In total 13,926 (unique CASE_IDs) records are present together in the year 2010 & 2016.
#########################
Overall statistics
YR 2010 = 6517 records and 
YR 2016 = 7409 (13.70% increase from YR 2010)
#########################
Collision Severity Codes: 
01 - Fatal
02 - Injury (Severe)
03 - Injury (Other Visible)
04 - Injury (Complaint of Pain)
00 - Property Damage Only (PDO)
###################################
Distribution of collission severity: 
############# YR 2010 #############
00 = 2839
01 = 29
02 = 195
03 = 1137
04 = 2317
###################################
############# YR 2016 #############
00 = 3351 (+19%)
01 = 35 (+20%)
02 = 273 (+40%)
03 = 1079 (+5%)
04 = 2671 (-13%)
###################################
Location information already present for
YR 2010 = 373 (5%)
YR 2016 = 1886 (25%)

"""
dfSWITRS2010=dfSWITRS[dfSWITRS["ACCIDENT_YEAR"].isin([2010])]
dfSWITRS2016=dfSWITRS[dfSWITRS["ACCIDENT_YEAR"].isin([2016])]

dfSWITRS2010.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Raw_SWITRS_2010.csv"))
dfSWITRS2016.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Raw_SWITR_2016.csv"))

c:\users\goyal\.virtualenvs\overpass_turbo-zwxzihl_\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# TIMS crash records summary
"""
There are two level of information available via TIMS website. 
One is for SF-City and other is SF-StateRoutes. 
They do not contain PDO crash information as it is available in the SWITRS file
"""
dfTIMSCity = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","TIMS_Crashes_city.csv"))
dfTIMSStateRoutes = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","TIMS_Crahes_StateRoutes.csv"))
# TIMS database
dfTIMSCity = dfTIMSCity[dfTIMSCity["ACCIDENT_YEAR"].isin([2010,2016])]
dfTIMSStateRoutes = dfTIMSStateRoutes[dfTIMSStateRoutes["ACCIDENT_YEAR"].isin([2010,2016])]
# YR 2010
dfTIMSCity2010 = dfTIMSCity[dfTIMSCity["ACCIDENT_YEAR"].isin([2010])]
dfTIMSStateRoutes2010 = dfTIMSStateRoutes[dfTIMSStateRoutes["ACCIDENT_YEAR"].isin([2010])]
# YR 2016
dfTIMSCity2016 = dfTIMSCity[dfTIMSCity["ACCIDENT_YEAR"].isin([2016])]
dfTIMSStateRoutes2016 = dfTIMSStateRoutes[dfTIMSStateRoutes["ACCIDENT_YEAR"].isin([2016])]

"""
########### RAW Stats ##################
SF-City 
YR 2010 = 3678
YR 2016 = 4058
SF-StateRoutes 
YR 2010 = 767
YR 2016 = 688
#######################################
Crash records in SF-StateRoutes for both YR 2010 & 2016 are already mentioned in the SF-City record, so use of the SF-StateRoutes information could be avoided.
Distribution of collission severity: 
############# YR 2010 #############
01 = 29
02 = 195
03 = 1137
04 = 2317
###################################
############# YR 2016 #############
01 = 35 (+20%)
02 = 273 (+40%)
03 = 1079 (+5%)
04 = 2671 (-13%)
###################################
Above information just indicates that except PDO crashes, the crash information 100% matches with SWITRS records.
###################################
Location information present for
YR 2010 = 3647 (99%)
YR 2016 = 3896 (96%)
###################################
Location information not present for
##### YR 2010 = 31
01 = 2
02 = 2
03 = 8
04 = 19
##### YR 2016 = 138
01 = 1
02 = 4
03 = 25
04 = 108
###################################

"""
dfTIMSCity2010.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Raw_TIMS_2010.csv"))
dfTIMSCity2016.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Raw_TIMS_2016.csv"))

# df2010 = dfSWITRS2010.merge(dfTIMSCity2010.drop_duplicates(), on=['CASE_ID','CASE_ID'], how='outer', indicator=True)
# df2016 = dfTIMSCity2016.merge(dfSWITRS2016.drop_duplicates(), on=['CASE_ID','CASE_ID'], how='left', indicator=True)

frames = [dfTIMSCity,dfSWITRS]
result_2 = pd.concat(frames)

result_2.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Raw_Switrs_TIMS_merged_all.csv"))

result_2['POINT_X']= np.where(result_2['POINT_X'].isnull(), result_2['LONGITUDE'], result_2['POINT_X'])
result_2['POINT_Y']= np.where(result_2['POINT_Y'].isnull(), result_2['LATITUDE'], result_2['POINT_Y'])

result_2['POINT_X']= np.where(result_2['POINT_X']>0, -abs(result_2['POINT_X']), result_2['POINT_X'])
result_2.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Raw_Switrs_TIMS_merged_all_v2.csv"))
result_3 = result_2.drop_duplicates(subset=['CASE_ID'])
result_3.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Raw_Switrs_TIMS_merged_all_v3.csv"))

# YR 2010
dfresult2010 = result_3[result_3["ACCIDENT_YEAR"].isin([2010])]
# YR 2016
dfresult2016 = result_3[result_3["ACCIDENT_YEAR"].isin([2016])]

dfresult2010.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Switrs_TIMS_merged_2010.csv"))
dfresult2016.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Switrs_TIMS_merged_2016.csv"))

"""
Once the merge is committed between SWITRS records and TIMS records, following are the statistics 
#############################################################
Total Crashes (captured)
YR 2010 = 6517
YR 2016 = 7409
#############################################################
Total number of latlong locations available
YR 2010 = 3860 (2657)
YR 2016 = 5057 (2352)
######## COLLISION SEVERITY based on LATLONG ################ 
FORMULA: TOTAL = with LATLONG + without LATLONG
YR 2010
00 - 2839 = 212 + 2627
01 - 29 = 27 + 2 
02 - 195 = 193 + 02
03 - 1137 = 1129 + 08
04 - 2317 = 2301 + 18

YR 2016
00 - 3351 = 1137 + 2214
01 - 35 = 34 + 1
02 - 273 = 269 + 4 
03 - 1079 = 1054 + 25
04 - 2671 = 2563 + 108
"""


c:\users\goyal\.virtualenvs\overpass_turbo-zwxzihl_\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (7,14,17,26,29,30,33,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


'\nOnce the merge is committed between SWITRS records and TIMS records, following are the statistics \n#############################################################\nTotal Crashes (captured)\nYR 2010 = 6517\nYR 2016 = 7409\n#############################################################\nTotal number of latlong locations available\nYR 2010 = 3860 (2657)\nYR 2016 = 5057 (2352)\n######## COLLISION SEVERITY based on LATLONG ################ \nFORMULA: TOTAL = with LATLONG + without LATLONG\nYR 2010\n00 - 2839 = 212 + 2627\n01 - 29 = 27 + 2 \n02 - 195 = 193 + 02\n03 - 1137 = 1129 + 08\n04 - 2317 = 2301 + 18\n\nYR 2016\n00 - 3351 = 1137 + 2214\n01 - 35 = 34 + 1\n02 - 273 = 269 + 4 \n03 - 1079 = 1054 + 25\n04 - 2671 = 2563 + 108\n'

In [4]:
dfTIMSCity = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","TIMS_Crashes_city.csv"))
dfTIMSStateRoutes = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","TIMS_Crahes_StateRoutes.csv"))
# TIMS database
dfTIMSCity = dfTIMSCity[dfTIMSCity["ACCIDENT_YEAR"].isin([2010,2016])]
dfTIMSStateRoutes = dfTIMSStateRoutes[dfTIMSStateRoutes["ACCIDENT_YEAR"].isin([2010,2016])]
# YR 2010
dfTIMSCity2010 = dfTIMSCity[dfTIMSCity["ACCIDENT_YEAR"].isin([2010])]
dfTIMSStateRoutes2010 = dfTIMSStateRoutes[dfTIMSStateRoutes["ACCIDENT_YEAR"].isin([2010])]
# YR 2016
dfTIMSCity2016 = dfTIMSCity[dfTIMSCity["ACCIDENT_YEAR"].isin([2016])]
dfTIMSStateRoutes2016 = dfTIMSStateRoutes[dfTIMSStateRoutes["ACCIDENT_YEAR"].isin([2016])]

c:\users\goyal\.virtualenvs\overpass_turbo-zwxzihl_\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (7,14,17,26,29,30,33,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
dfresult2016['POINT_X'].isnull().value_counts()

False    5057
True     2352
Name: POINT_X, dtype: int64

In [6]:
###### REVERSE GEOCODE ########
BASE_DIR = Path.cwd()
dfIntersect_Rds = pd.read_csv(BASE_DIR.parent.joinpath("Exported_files","Intersecting_Rds.csv"))
dfIntersect_Rds['geometry'] = dfIntersect_Rds['geometry'].apply(wkt.loads)
dfIntersect_Rds = gpd.GeoDataFrame(dfIntersect_Rds, crs='epsg:4326')
col = ["Unnamed: 0", "index","layer","path"]
dfIntersect_Rds = dfIntersect_Rds.drop(col,axis=1).copy()

def cleanStreetName(streetName):
    newStreetName = streetName.strip()
    corrections = {"TWELFTH":"12TH", 
                   "ELEVENTH":"11TH",
                   "TENTH":"10TH",
                   "NINTH":"9TH",
                   "EIGHTH":"8TH",
                   "SEVENTH":"7TH",
                   "SIXTH":"6TH",
                   "FIFTH":"5TH",
                   "FOURTH":"4TH",
                   "THIRD":"3RD",
                   "SECOND":"2ND",
                   "FIRST":"1ST",
                   "O'FARRELL":"O FARRELL",
                   "3RDREET":"3RD",
                   "EMBARCADERO/KING":"THE EMBARCADERO",
                   "VAN NESSNUE":"VAN NESS",
                   "3RD #3":"3RD",
                   "BAYSHORE #3":"BAYSHORE",
                   "09TH":"9TH",
                   "08TH":"8TH",
                   "07TH":"7TH",
                   "06TH":"6TH",
                   "05TH":"5TH",
                   "04TH":"4TH",
                   "03RD":"3RD",
                   "02ND":"2ND",
                   "01ST":"1ST",
                  }

    itemsToRemove = [" STREETS",
                     " STS.",
                     " STS",
                     " ST.",
                     " ST",
                     " Street",
                     " RD.",
                     " RD",
                     " Road",
                     " AVE.",
                     " AVES",
                     " AVE",
                     " AV",
                     " Avenue",
                     " BLVD.",
                     " BLVD",
                     " BL",                     
                     " Boulevard",
                     " MASTER:",
                     " DR.",
                     " Drive",
                     " WY",
                     " Way",
                     " CT",
                     " TERR",
                     " Terrace",
                     " HWY",
                     " EXPY"]
          
    for wrongName, rightName in corrections.items():
        if wrongName in streetName:
            newStreetName = streetName.replace(wrongName, rightName)
        if streetName == 'EMBARCADERO':
            newStreetName = "THE EMBARCADERO"
        if streetName.endswith(" DR"):
            newStreetName = streetName[:-3]
        if streetName.endswith(" AV"):
            newStreetName = streetName[:-3]
        if " TO " in streetName:
            cutOff = streetName.find(" TO ")
            newStreetName = streetName[:cutOff]            
    
    for item in itemsToRemove:
        if item in newStreetName:
            newStreetName = newStreetName.replace(item, "")
    
    return newStreetName.strip()

dfIntersect_Rds=dfIntersect_Rds.fillna("").copy()

dfIntersect_Rds["Link_ID_0_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_0"]), axis=1)
dfIntersect_Rds["Link_ID_1_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_1"]), axis=1)
dfIntersect_Rds["Link_ID_2_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_2"]), axis=1)
dfIntersect_Rds["Link_ID_3_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_3"]), axis=1)
dfIntersect_Rds["Link_ID_4_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_4"]), axis=1)
dfIntersect_Rds["Link_ID_5_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_5"]), axis=1)
dfIntersect_Rds["Link_ID_6_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_6"]), axis=1)
dfIntersect_Rds["Link_ID_7_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_7"]), axis=1)

dfIntersect_Rds['Link_Names'] = dfIntersect_Rds[['Link_ID_0_2', 'Link_ID_1_2','Link_ID_2_2', 'Link_ID_3_2','Link_ID_4_2', 'Link_ID_5_2','Link_ID_6_2', 'Link_ID_7_2']].apply(lambda x: ','.join(set(x.unique())), axis=1)

# remove first character from the values if it starts with ","
dfIntersect_Rds['Link_Names'] = dfIntersect_Rds['Link_Names'].apply(lambda x : x[1:] if x.startswith(",") else x)

# lowercase
dfIntersect_Rds = dfIntersect_Rds.applymap(lambda s:s.lower() if type(s) == str else s)

dfIntersect_Rds["LatLong"] = dfIntersect_Rds["lat"].astype(str) + " " + dfIntersect_Rds["lon"].astype(str)
dfIntersect_Rds.to_csv(BASE_DIR.parent.joinpath("Exported_Files","Intersecting_Rds_2.csv"))

# get unique street intersections and their latitude and longtitude
dfunique = dfIntersect_Rds[['Link_Names', 'LatLong']]
# get unique street intersections and their latitude and longtitude
dfunique.drop_duplicates(['Link_Names'])
mydict = dict(zip(dfunique.Link_Names, dfunique.LatLong.astype(str)))

from shapely.geometry import Point

def getDerivedLatLngPosition(lat,long, dist, bearing):
    latitude = float(lat)
    longitude = float(long)
    
    DegreesToRadians = math.pi/180.0
    RadiansToDegrees = 180.0/math.pi 
    EarthRadius = 6378137.0 # Radius of Earth in meters

    latA =  latitude * DegreesToRadians
    longA = longitude * DegreesToRadians
    angularDistance = dist/EarthRadius
    trueCourse = bearing * DegreesToRadians
    
    lat =  math.asin(math.sin(latA) * math.cos(angularDistance) + math.cos(latA) * math.sin(angularDistance) * math.cos(trueCourse))
    
    dlon = math.atan2(math.sin(trueCourse) * math.sin(angularDistance) * math.cos(latA), math.cos(angularDistance) - math.sin(latA) * math.sin(lat))
    lon = ((longA + dlon + math.pi) % (math.pi*2)) - math.pi
    
    return Point(round(lon * RadiansToDegrees,5),round(lat * RadiansToDegrees,5))

# Switcher is dictionary data type here
def get_direction(argument):
    switcher = {
        "N": 0,
        "W": -90,
        "E": +90,
        "S": -180
    }  
    # get() method of dictionary data type returns value of passed argument if it is present in dictionary;
    # otherwise second argument will be assigned as default value of passed argument
    return switcher.get(argument, 0)

def latlongsearch(row):
    primary_rd = row["PRIMARY_RD_2"]
    secondary_rd = row["SECONDARY_RD_2"] 
    _dict = mydict.copy()
    for key, value in _dict.items():
        if (primary_rd in key) and (secondary_rd in key):
            lat = value.split(" ")[0]
            long = value.split(" ")[1]
            dist = 0 if (row["DISTANCE"]<= 0) else (row["DISTANCE"]/3.2808) # convert distance into meters
#             print(row["DIRECTION"])
            bearing = get_direction(row["DIRECTION"].upper())
            return getDerivedLatLngPosition(lat,long, dist, bearing)
    return None

In [7]:
# YR 2010
dfresult2010.columns = dfresult2010.columns.str.rstrip('_x')  # strip suffix at the right end only.
dfresult2010["PRIMARY_RD_2"] = dfresult2010.apply(lambda x: cleanStreetName(x["PRIMARY_RD"]), axis=1)
dfresult2010["SECONDARY_RD_2"] = dfresult2010.apply(lambda x: cleanStreetName(x["SECONDARY_RD"]), axis=1)
# dfSWITRS['Rd_Names'] = dfSWITRS[['PRIMARY_RD_2', 'SECONDARY_RD_2']].apply(lambda x: ' '.join(x), axis=1)
dfresult2010['Rd_Names'] = dfresult2010[['PRIMARY_RD_2', 'SECONDARY_RD_2']].apply(lambda x: ','.join(set(x.unique())), axis=1)
# remove first character from the values if it starts with ","
dfresult2010['Rd_Names'] = dfresult2010['Rd_Names'].apply(lambda x : x[1:] if x.startswith(",") else x)
# lowercase
dfresult2010 = dfresult2010.applymap(lambda s:s.lower() if type(s) == str else s)

dfresult2010["DIRECTION"]=dfresult2010["DIRECTION"].fillna('')
dfresult2010["ref_latlong"] = dfresult2010.apply(lambda row: latlongsearch(row), axis=1)

<ipython-input-7-330666bc9c2e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfresult2010["PRIMARY_RD_2"] = dfresult2010.apply(lambda x: cleanStreetName(x["PRIMARY_RD"]), axis=1)
<ipython-input-7-330666bc9c2e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfresult2010["SECONDARY_RD_2"] = dfresult2010.apply(lambda x: cleanStreetName(x["SECONDARY_RD"]), axis=1)
<ipython-input-7-330666bc9c2e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [8]:
# YR 2016
dfresult2016.columns = dfresult2016.columns.str.rstrip('_x')  # strip suffix at the right end only.
dfresult2016["PRIMARY_RD_2"] = dfresult2016.apply(lambda x: cleanStreetName(x["PRIMARY_RD"]), axis=1)
dfresult2016["SECONDARY_RD_2"] = dfresult2016.apply(lambda x: cleanStreetName(x["SECONDARY_RD"]), axis=1)
# dfSWITRS['Rd_Names'] = dfSWITRS[['PRIMARY_RD_2', 'SECONDARY_RD_2']].apply(lambda x: ' '.join(x), axis=1)
dfresult2016['Rd_Names'] = dfresult2016[['PRIMARY_RD_2', 'SECONDARY_RD_2']].apply(lambda x: ','.join(set(x.unique())), axis=1)
# remove first character from the values if it starts with ","
dfresult2016['Rd_Names'] = dfresult2016['Rd_Names'].apply(lambda x : x[1:] if x.startswith(",") else x)
# lowercase
dfresult2016 = dfresult2016.applymap(lambda s:s.lower() if type(s) == str else s)

dfresult2016["DIRECTION"]=dfresult2016["DIRECTION"].fillna('')
dfresult2016["ref_latlong"] = dfresult2016.apply(lambda row: latlongsearch(row), axis=1)

<ipython-input-8-d44de6d9133e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfresult2016["PRIMARY_RD_2"] = dfresult2016.apply(lambda x: cleanStreetName(x["PRIMARY_RD"]), axis=1)
<ipython-input-8-d44de6d9133e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfresult2016["SECONDARY_RD_2"] = dfresult2016.apply(lambda x: cleanStreetName(x["SECONDARY_RD"]), axis=1)
<ipython-input-8-d44de6d9133e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [9]:
def fill_null_latlong(row):
    if pd.isnull(row["ref_latlong"]):
        if pd.notna(row["POINT_X"]):
            return Point(round(row["POINT_X"],5),round(row["POINT_Y"],5))
    else:
        return row["ref_latlong"]
    
dfresult2010["ref_latlong"] = dfresult2010.apply(lambda row: fill_null_latlong(row), axis=1)        
dfresult2016["ref_latlong"] = dfresult2016.apply(lambda row: fill_null_latlong(row), axis=1)    

In [10]:
dfresult2010["ref_latlong"].isnull().value_counts()

False    5322
True     1195
Name: ref_latlong, dtype: int64

In [11]:
dfresult2010[dfresult2010["ref_latlong"].isnull()==False]["COLLISION_SEVERITY"].value_counts()

4    2301
0    1671
3    1129
2     194
1      27
Name: COLLISION_SEVERITY, dtype: int64

In [12]:
# ###################################
# Distribution of collission severity: 
# ############# YR 2010 #############
# 00 = 2839
# 01 = 29
# 02 = 195
# 03 = 1137
# 04 = 2317
# ###################################
# ############# YR 2016 #############
# 00 = 3351 (+19%)
# 01 = 35 (+20%)
# 02 = 273 (+40%)
# 03 = 1079 (+5%)
# 04 = 2671 (-13%)
# ###################################

In [13]:
'''
After merging the SWITRS and TIMS Data and performing reverse geocode following are the stats
#############################################################
Total Crashes (captured)
YR 2010 = 6517
YR 2016 = 7409
###########################################
Total number of latlong locations available
YR 2010 = 5322 (1195)
YR 2016 = 6672 (737)
######## COLLISION SEVERITY based on LATLONG ################
YR 2010
00 - 4511 = 1671  (2840)
01 - 29 = 27 + 02 
02 - 195 = 194 + 01
03 - 1137 = 1129 + 08
04 - 2317 = 2301 + 16

YR 2016
00 - 3351 = 2752 (599)
01 - 35 = 34 + 1
02 - 273 = 269 + 4
03 - 1079 = 1054 + 25
04 - 2671 = 2563 + 108

'''

'\nAfter merging the SWITRS and TIMS Data and performing reverse geocode following are the stats\n#############################################################\nTotal Crashes (captured)\nYR 2010 = 6517\nYR 2016 = 7409\n###########################################\nTotal number of latlong locations available\nYR 2010 = 5322 (1195)\nYR 2016 = 6672 (737)\n######## COLLISION SEVERITY based on LATLONG ################\nYR 2010\n00 - 4511 = 1671  (2840)\n01 - 29 = 27 + 02 \n02 - 195 = 194 + 01\n03 - 1137 = 1129 + 08\n04 - 2317 = 2301 + 16\n\nYR 2016\n00 - 3351 = 2752 (599)\n01 - 35 = 34 + 1\n02 - 273 = 269 + 4\n03 - 1079 = 1054 + 25\n04 - 2671 = 2563 + 108\n\n'

In [14]:
dfresult2010.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_2010_SWITRS_TIMS_matched_output.csv"))
dfresult2016.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_2016_SWITRS_TIMS_matched_output.csv"))

In [15]:
dfresult = pd.concat([dfresult2010, dfresult2016]).to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_all_SWITRS_TIMS_matched_output.csv"))

In [16]:
###### trying to refine the geocoding #########

In [2]:
dfIntersections = pd.read_csv(BASE_DIR.parent.joinpath("TransBase_Shapefiles","street_intersections.csv"))
dfStreetNames = pd.read_csv(BASE_DIR.parent.joinpath("TransBase_Shapefiles","Street_Names.csv"))
# dfIntersections["x_street_comb"].str.split(pat="/",expand=True).rename(columns = lambda x: f"StreetName_{x+1}")
# dfIntersections["x_street_comb"].str.split(pat="/",expand=True).rename(columns = lambda x: f"StreetName_{x+1}")
dfIntersections[["StreetName_1","StreetName_2","StreetName_3","StreetName_4","StreetName_5"]] = dfIntersections["x_street_comb"].str.split(pat="/",expand=True)

dfIntersections = dfIntersections[dfIntersections["intrsctn_type"]=="INTERSECTIONS"]
dfIntersections.to_csv(BASE_DIR.parent.joinpath("TransBase_Shapefiles","street_intersections_split.csv"))


dfresult2010 = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_2010_SWITRS_TIMS_matched_output.csv"))
dfresult2016 = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_2016_SWITRS_TIMS_matched_output.csv"))

dfresult = pd.concat([dfresult2010, dfresult2016]).to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_all_SWITRS_TIMS_matched_output.csv"))

In [3]:
# get the unique street names in TransBase Shapefile
unique_streetNames = pd.unique(dfIntersections[["StreetName_1","StreetName_2","StreetName_3","StreetName_4","StreetName_5"]].values.ravel('K'))

In [4]:
dfUniqueStreets = pd.DataFrame(unique_streetNames)
dfUniqueStreets.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Unique_Street_Names.csv"))

In [5]:
def cleanStreetName(streetName):
    newStreetName = streetName.strip()
    corrections = {"TWELFTH":"12TH", 
                   "ELEVENTH":"11TH",
                   "TENTH":"10TH",
                   "NINTH":"9TH",
                   "EIGHTH":"8TH",
                   "SEVENTH":"7TH",
                   "SIXTH":"6TH",
                   "FIFTH":"5TH",
                   "FOURTH":"4TH",
                   "THIRD":"3RD",
                   "SECOND":"2ND",
                   "FIRST":"1ST",
                   "O'FARRELL":"O FARRELL",
                   "3RDREET":"3RD",
                   "EMBARCADERO/KING":"THE EMBARCADERO",
                   "VAN NESSNUE":"VAN NESS",
                   "3RD #3":"3RD",
                   "BAYSHORE #3":"BAYSHORE",
                   "09TH":"9TH",
                   "08TH":"8TH",
                   "07TH":"7TH",
                   "06TH":"6TH",
                   "05TH":"5TH",
                   "04TH":"4TH",
                   "03RD":"3RD",
                   "02ND":"2ND",
                   "01ST":"1ST",
                  }

    itemsToRemove = [" STREETS",
                     " STS.",
                     " STS",
                     " ST.",
                     " ST",
                     " Street",
                     " STREET",
                     " RD.",
                     " RD",
                     " Road",
                     " AVE.",
                     " AVES",
                     " AVE",
                     " AV",
                     " Avenue",
                     " BLVD.",
                     " BLVD",
                     " BL",                     
                     " Boulevard",
                     " MASTER:",
                     " DR.",
                     " DR",
                     " Drive",
                     " WY",
                     " Way",
                     " CT",
                     " TERR",
                     " Terrace",
                     " HWY",
                     " EXPY",
                    ]
          
    for wrongName, rightName in corrections.items():
        if wrongName in streetName:
            newStreetName = streetName.replace(wrongName, rightName)
        if streetName == 'EMBARCADERO':
            newStreetName = "THE EMBARCADERO"
        if streetName.endswith(" DR"):
            newStreetName = streetName[:-3]
        if streetName.endswith(" AV"):
            newStreetName = streetName[:-3]
        if " TO " in streetName:
            cutOff = streetName.find(" TO ")
            newStreetName = streetName[:cutOff]            
    
    for item in itemsToRemove:
        if item in newStreetName:
            newStreetName = newStreetName.replace(item, "")
    
    return newStreetName.strip()

# dfIntersections["Link_ID_1"] = dfIntersections.apply(lambda x: cleanStreetName(x["StreetName_1"]), axis=1)
# dfIntersections["Link_ID_2"] = dfIntersections.apply(lambda x: cleanStreetName(x["StreetName_2"]), axis=1)
# dfIntersections["Link_ID_3"] = dfIntersections.apply(lambda x: cleanStreetName(x["StreetName_3"]), axis=1)
# dfIntersections["Link_ID_4"] = dfIntersections.apply(lambda x: cleanStreetName(x["StreetName_4"]), axis=1)
# dfIntersections["Link_ID_5"] = dfIntersections.apply(lambda x: cleanStreetName(x["StreetName_5"]), axis=1)


In [16]:
dfresult=pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_all_SWITRS_TIMS_matched_output.csv"))

orig_list = [" s/b"," n/b"," e/b"," w/b",
             "us 101","us-101","rt 101",
            " AV", " AVENUE",
            " STREET", 
             " BL", 
             " WY",
            "BAYSHORE ", "BAYSHORE", "BAYSHORE AVE", "BAYSHORE BLVD",
             "bernal hts ", " tunnel", 
             "buena vista west", "buena vista west","buena vista east","buena vista av",
             "chain of lakes dr e", 
             "martin luther king dr", "martin luther king jr",
             "columbia st",
             "conservatory dr east",
             " BY PASS",
             "e/b i-80", "eb i-80",
             "nb us-101",
             "embarcadero north", "embarcadero south", "embarcadero st",
             "i- 80", " (sfobb)", "(n/b)", "u/c", " o/c"
             " (1500 block)"," 1600 block", " (blk 20)",
             "rt 1", "rt 80","rt 280", "sfobb",
             "s/b i-280", " to 18th street","s/b us-101",
             " 4100", " (rt35)","southbound us-101 to san bruno ave.","sr 1 (s/b)", "sr-1"," to us-101 n/b",
             " 200 block", "onramp", " to i-80 Eastbound", " (golden gate bridge)", "u.s.","united states highway",
             " (ggb)","(ggb)", " (n/b)", " (s/b)", " (e/b)", " (w/b)"," from cesar chavez st"," from merchant rd.", " to 9th st.",
             " to i-280 s/b", " to i-80 e/b", " from 10th street","(ggb west sidewalk)"," at i-280 split", 
             " from bayshore blvd", " from cesar chavez st on-ramp", "from cesar chavez st.", " to 9th street", "  to alemany blvd.", " to bayshore blvd.",
             " to cesar chavez"," from cesar chavez st,"," from cesar chavez st", " to duboce avenue"," to i-280",
             " to mission st.", " to mission st"," to mission street", " to octavia blvd."," to silver avenue", " to us-101", " to vermont st", " to vermont street",
             " golden gate bridge", " overpass", " under-crossing", " under crossing", " undercrossing", "faith street pedestrian"," ped",
             " (doyle dr.)"," (ggb east side walk)", " (ggb west sidewalk)", "  from 10th st.", "  from 10th st", " from oyster point blvd", "  from s. van ness ave."," from s. van ness", " oc",
             " to alemany blvd.","  to bayshore boulevard", ". east","  to paul avenue", " transition rd", " to westbound", " from van ness ave", " sb", " silver ave. off ramp",
             "  from 10th street"," from merchant road", "1101", " 131",
            ]
repl_list = [" Southbound"," Northbound"," Eastbound", " Westbound",
             "HWY 101","HWY 101","HWY 101",
            " AVE"," AVE",
            " ST", " BLVD", " WAY",
            "BAY SHORE BLVD", "BAY SHORE BLVD","BAY SHORE BLVD","BAY SHORE BLVD",
            "BERNAL HEIGHTS ", "",
             "BUENA VISTA WEST AVE", "BUENA VISTA EAST AVE","BUENA VISTA WEST AVE",
             "chain of lakes dr", 
             "MARTIN LUTHER KING JR DR", "MARTIN LUTHER KING JR DR",
             "COLUMBIA SQUARE ST",
             "CONSERVATORY EAST DR",
             " BYPASS",
              "I-80 EASTBOUND", "I-80 EASTBOUND",
             "HWY 101 Northbound",
             "THE EMBARCADERO", "THE EMBARCADERO", "THE EMBARCADERO",
             "I-80","", "Northbound", "", "",
             "","","",
             "HWY 1", "I-80","I-280", "",
             "I-280 Southbound", "","HWY 101 Southbound",
             "", "","HWY 101 Southbound", "HWY 1 Southbound", "HWY 1", "",
             "", "ON RAMP","","","HWY", "HWY.",
             "","","","","","","","","",
             "","", "", "", "", 
             "" , "", "","", "", "", "",
             "", "", "","", "",
             "", "", "", "", "","","","",
             "", "", "", "", "", "", "",
             "", "", "", "","", "", "","","",
             "", "", "", "","", "","","","",
             "", "", "",   
            ]

dfresult[["PRIMARY_RD_3","SECONDARY_RD_3"]] = dfresult[["PRIMARY_RD_2","SECONDARY_RD_2"]].replace(orig_list,repl_list,regex=True)

In [17]:
dfresult["PRIMARY_RD_3"] = dfresult["PRIMARY_RD_3"].str.upper()
dfresult["SECONDARY_RD_3"] = dfresult["SECONDARY_RD_3"].str.upper()
dfresult["PRIMARY_RD_3"] = dfresult.apply(lambda x: cleanStreetName(x["PRIMARY_RD_3"]), axis=1)
dfresult["SECONDARY_RD_3"] = dfresult.apply(lambda x: cleanStreetName(x["SECONDARY_RD_3"]), axis=1)

In [18]:
dfIntersections["Intersecting_Streets"] = dfIntersections.apply(lambda x: cleanStreetName(x["x_street_comb"]), axis=1)

In [19]:
dfIntersections["LatLong"] = dfIntersections["latitude"].astype(str) + " " + dfIntersections["longitude"].astype(str)
# get unique street intersections and their latitude and longtitude
dfIntersections["Intersecting_Streets"] = dfIntersections["Intersecting_Streets"].str.upper()
dfIntersections.drop_duplicates(['Intersecting_Streets'])
mydict = dict(zip(dfIntersections.Intersecting_Streets, dfIntersections.LatLong.astype(str)))

In [20]:
from shapely.geometry import Point

def getDerivedLatLngPosition(lat,long, dist, bearing):
    latitude = float(lat)
    longitude = float(long)
    
    DegreesToRadians = math.pi/180.0
    RadiansToDegrees = 180.0/math.pi 
    EarthRadius = 6378137.0 # Radius of Earth in meters

    latA =  latitude * DegreesToRadians
    longA = longitude * DegreesToRadians
    angularDistance = dist/EarthRadius
    trueCourse = bearing * DegreesToRadians
    
    lat =  math.asin(math.sin(latA) * math.cos(angularDistance) + math.cos(latA) * math.sin(angularDistance) * math.cos(trueCourse))
    
    dlon = math.atan2(math.sin(trueCourse) * math.sin(angularDistance) * math.cos(latA), math.cos(angularDistance) - math.sin(latA) * math.sin(lat))
    lon = ((longA + dlon + math.pi) % (math.pi*2)) - math.pi
    
    return Point(round(lon * RadiansToDegrees,5),round(lat * RadiansToDegrees,5))

# Switcher is dictionary data type here
def get_direction(argument):
    switcher = {
        "N": 0,
        "W": -90,
        "E": +90,
        "S": -180
    }  
    # get() method of dictionary data type returns value of passed argument if it is present in dictionary;
    # otherwise second argument will be assigned as default value of passed argument
    return switcher.get(argument, 0)

In [23]:
def latlongsearch(row):
    primary_rd = row["PRIMARY_RD_3"]
    secondary_rd = row["SECONDARY_RD_3"] 
    _dict = mydict.copy()
    for key, value in _dict.items():
        if (primary_rd in key) and (secondary_rd in key):
            lat = value.split(" ")[0]
            long = value.split(" ")[1]
            dist = 0 if (row["DISTANCE"]<= 0) else (row["DISTANCE"]/3.2808)
#             print(row["DIRECTION"])
            bearing = get_direction(row["DIRECTION"].upper())
            return getDerivedLatLngPosition(lat,long, dist, bearing)
    return None         
dfresult["DIRECTION"]=dfresult["DIRECTION"].fillna('')
dfresult["ref_latlong2"] = dfresult.apply(lambda row: latlongsearch(row), axis=1)
dfresult.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_SWITRS_TIMS_matched_output_v3.csv"))

In [3]:
dfresult

,Unnamed: 0,Unnamed: 0.1,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,...,COUNTY,CITY,POINT_X,POINT_Y,PRIMARY_RD_2,SECONDARY_RD_2,Rd_Names,ref_latlong,PRIMARY_RD_3,SECONDARY_RD_3
0,0,0,4392133,2010,2012-01-23,9335,2010-01-31,415,19084,NaN,...,san francisco,san francisco,-122.405503,37.775845,rt 80,7th,"7th,rt 80",POINT (-122.4055 37.77584),I-80,7th st
1,1,1,4392442,2010,2012-01-25,9335,2010-02-25,1302,17672,NaN,...,san francisco,san francisco,-122.428798,37.732164,rt 280,mission,"rt 280,mission",POINT (-122.4288 37.73216),I-280,mission st
2,2,2,4523135,2010,2010-10-26,3801,2010-01-07,841,1794,south,...,san francisco,san francisco,-122.410433,37.778795,7th,minna,"7th,minna",POINT (-122.41043 37.77879),7th st,minna st
3,3,3,4523139,2010,2010-10-26,3801,2010-01-05,3,307,bayvi,...,san francisco,san francisco,-122.401383,37.762285,de haro,18th,"de haro,18th",POINT (-122.40133 37.76228),de haro st,18th st
4,4,4,4523186,2010,2010-10-26,3801,2010-01-09,545,000779,north,...,san francisco,san francisco,-122.417583,37.783305,larkin,eddy,"larkin,eddy",POINT (-122.41755 37.78332),larkin st,eddy st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13921,7404,44824,90366351,2016,20170111,9335,20161114,1800,020509,NaN,...,NaN,NaN,-122.428880,37.732190,i-280 southbound,missionreet o/c,"i-280 southbound,missionreet o/c",POINT (-122.42888 37.73219),i-280,mission st
13922,7405,44826,90371177,2016,20170213,9335,20161216,1735,021424,NaN,...,NaN,NaN,-122.406490,37.743290,us-101 s/b,cesar chavez,"cesar chavez,us-101 s/b",POINT (-122.40649 37.74329),HWY 101,cesar chavez st
13923,7406,44827,90373242,2016,20170119,9335,20161208,1550,021293,NaN,...,NaN,NaN,-122.405750,37.767930,us-101 s/b,alameda,"us-101 s/b,alameda",POINT (-122.40575 37.76793),HWY 101,alameda st
13924,7407,44828,90378461,2016,20170126,9335,20161203,1750,021455,NaN,...,NaN,NaN,-122.403710,37.749020,us-101 n/b,cesar chavez,"cesar chavez,us-101 n/b",POINT (-122.40371 37.74902),HWY 101,cesar chavez st


In [2]:
dfIntersections = pd.read_csv(BASE_DIR.parent.joinpath("TransBase_Shapefiles","street_intersections.csv"))

dfresult=pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_all_SWITRS_TIMS_matched_output.csv"))

dfrectified = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_SWITRS_TIMS_matched_output_v3.3.csv"))
PRIMARY_RD_dict = dict(zip(dfrectified["CASE_ID"],dfrectified["PRIMARY_RD_3"]))
SECONDARY_RD_dict = dict(zip(dfrectified["CASE_ID"],dfrectified["SECONDARY_RD_3"]))
dfresult["PRIMARY_RD_3"] = dfresult["CASE_ID"].map(PRIMARY_RD_dict)
dfresult["SECONDARY_RD_3"] = dfresult["CASE_ID"].map(SECONDARY_RD_dict)
dfresult = dfresult.dropna(axis=0, subset=["PRIMARY_RD_3","SECONDARY_RD_3"])

In [3]:
def cleanStreetName(streetName):
    newStreetName = streetName.strip()
    corrections = {"TWELFTH":"12TH", 
                   "ELEVENTH":"11TH",
                   "TENTH":"10TH",
                   "NINTH":"9TH",
                   "EIGHTH":"8TH",
                   "SEVENTH":"7TH",
                   "SIXTH":"6TH",
                   "FIFTH":"5TH",
                   "FOURTH":"4TH",
                   "THIRD":"3RD",
                   "SECOND":"2ND",
                   "FIRST":"1ST",
                   "3RDREET":"3RD",
                   "EMBARCADERO/KING":"THE EMBARCADERO",
                   "VAN NESSNUE":"VAN NESS",
                   "3RD #3":"3RD",
                   "09TH":"9TH",
                   "08TH":"8TH",
                   "07TH":"7TH",
                   "06TH":"6TH",
                   "05TH":"5TH",
                   "04TH":"4TH",
                   "03RD":"3RD",
                   "02ND":"2ND",
                   "01ST":"1ST",
                  }

    itemsToRemove = [" STREETS",
                     " STS.",
                     " STS",
                     " ST.",
                     " ST",
                     " Street",
                     " STREET",
                     " RD.",
                     " RD",
                     " Road",
                     " AVE.",
                     " AVES",
                     " AVE",
                     " AV",
                     " Avenue",
                     " BLVD.",
                     " BLVD",
                     " BL",                     
                     " Boulevard",
                     " MASTER:",
                     " DR.",
                     " DR",
                     " Drive",
                     " WY",
                     " Way",
                     " CT",
                     " TERR",
                     " Terrace",
                     " EXPY",
                    ]
          
    for wrongName, rightName in corrections.items():
        if wrongName in streetName:
            newStreetName = streetName.replace(wrongName, rightName)
        if streetName == 'EMBARCADERO':
            newStreetName = "THE EMBARCADERO"
        if streetName.endswith(" DR"):
            newStreetName = streetName[:-3]
        if streetName.endswith(" AV"):
            newStreetName = streetName[:-3]
        if " TO " in streetName:
            cutOff = streetName.find(" TO ")
            newStreetName = streetName[:cutOff]            
    
    for item in itemsToRemove:
        if item in newStreetName:
            newStreetName = newStreetName.replace(item, "")
    
    return newStreetName.strip()

dfresult["PRIMARY_RD_3"] = dfresult["PRIMARY_RD_3"].str.upper()
dfresult["SECONDARY_RD_3"] = dfresult["SECONDARY_RD_3"].str.upper()
dfresult["PRIMARY_RD_3"] = dfresult.apply(lambda x: cleanStreetName(x["PRIMARY_RD_3"]), axis=1)
dfresult["SECONDARY_RD_3"] = dfresult.apply(lambda x: cleanStreetName(x["SECONDARY_RD_3"]), axis=1)

In [4]:
dfIntersections["LatLong"] = dfIntersections["latitude"].astype(str) + " " + dfIntersections["longitude"].astype(str)
# get unique street intersections and their latitude and longtitude
dfIntersections["Intersecting_Streets"] = dfIntersections["x_street_comb"].str.upper()
dfIntersections.drop_duplicates(['Intersecting_Streets'])
mydict = dict(zip(dfIntersections.Intersecting_Streets, dfIntersections.LatLong.astype(str)))

In [5]:
from shapely.geometry import Point

def getDerivedLatLngPosition(lat,long, dist, bearing):
    latitude = float(lat)
    longitude = float(long)
    
    DegreesToRadians = math.pi/180.0
    RadiansToDegrees = 180.0/math.pi 
    EarthRadius = 6378137.0 # Radius of Earth in meters

    latA =  latitude * DegreesToRadians
    longA = longitude * DegreesToRadians
    angularDistance = dist/EarthRadius
    trueCourse = bearing * DegreesToRadians
    
    lat =  math.asin(math.sin(latA) * math.cos(angularDistance) + math.cos(latA) * math.sin(angularDistance) * math.cos(trueCourse))
    
    dlon = math.atan2(math.sin(trueCourse) * math.sin(angularDistance) * math.cos(latA), math.cos(angularDistance) - math.sin(latA) * math.sin(lat))
    lon = ((longA + dlon + math.pi) % (math.pi*2)) - math.pi
    
    return Point(round(lon * RadiansToDegrees,5),round(lat * RadiansToDegrees,5))

# Switcher is dictionary data type here
def get_direction(argument):
    switcher = {
        "N": 0,
        "W": -90,
        "E": +90,
        "S": -180
    }  
    # get() method of dictionary data type returns value of passed argument if it is present in dictionary;
    # otherwise second argument will be assigned as default value of passed argument
    return switcher.get(argument, 0)


def latlongsearch(row):
    primary_rd = row["PRIMARY_RD_3"]
    secondary_rd = row["SECONDARY_RD_3"] 
    _dict = mydict.copy()
    for key, value in _dict.items():
        if (primary_rd in key) and (secondary_rd in key):
            lat = value.split(" ")[0]
            long = value.split(" ")[1]
            dist = 0 if (row["DISTANCE"]<= 0) else (round((row["DISTANCE"]/3.2808)*0.1,2))
#             print(row["DIRECTION"])
            bearing = get_direction(row["DIRECTION"].upper())
            return getDerivedLatLngPosition(lat,long, dist, bearing)
    return None         
dfresult["DIRECTION"]=dfresult["DIRECTION"].fillna('')
dfresult["ref_latlong2"] = dfresult.apply(lambda row: latlongsearch(row), axis=1)
dfresult.to_csv(BASE_DIR.parent.joinpath("Data","22Oct2021","Road_Crashes_SWITRS_TIMS_matched_output_v3.4.csv"))

In [6]:
def fill_null_latlong(row):
    if pd.isnull(row["ref_latlong2"]):
        if pd.notna(row["POINT_X"]):
            return Point(round(row["POINT_X"],5),round(row["POINT_Y"],5))
    else:
        return row["ref_latlong2"]
dfresult["ref_latlong2"] = dfresult.apply(lambda row: fill_null_latlong(row), axis=1)
cols = ["Unnamed: 0","Unnamed: 0.1","PRIMARY_RD_2","SECONDARY_RD_2","Rd_Names","ref_latlong"]
dfresult = dfresult.drop(cols,axis=1).copy()
dfresult.to_csv(BASE_DIR.parent.joinpath("Data","22Oct2021","Road_Crashes_SWITRS_TIMS_matched_output_v3.4.csv"))

In [29]:
df2010 = dfresult[dfresult["ACCIDENT_YEAR"]==2010]
df2016 = dfresult[dfresult["ACCIDENT_YEAR"]==2016]

# find the null cells
dfcaseID2010 = df2010[df2010["ref_latlong2"].isnull()]["CASE_ID"].to_list()
dfcaseID2016 = df2016[df2016["ref_latlong2"].isnull()]["CASE_ID"].to_list()

rd_crash_2010 = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Edited","2010_Road_Crashes_SWITRS_TIMS_matched_output_v3.4.csv"))
rd_crash_2016 = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Edited","2016_Road_Crashes_SWITRS_TIMS_matched_output_v3.4.csv"))

lstcaseID2010 = dict(zip(rd_crash_2010[rd_crash_2010["CASE_ID"].isin(dfcaseID2010)]["CASE_ID"],rd_crash_2010[rd_crash_2010["CASE_ID"].isin(dfcaseID2010)]["ref_latlong2"]))
lstcaseID2016 = dict(zip(rd_crash_2016[rd_crash_2016["CASE_ID"].isin(dfcaseID2016)]["CASE_ID"],rd_crash_2016[rd_crash_2016["CASE_ID"].isin(dfcaseID2016)]["ref_latlong2"]))

df2010["ref_latlong2"] = df2010["CASE_ID"].map(lstcaseID2010).fillna(df2010["ref_latlong2"])
df2016["ref_latlong2"] = df2016["CASE_ID"].map(lstcaseID2016).fillna(df2016["ref_latlong2"])
    
# df2010 = dfresult[dfresult["ACCIDENT_YEAR"]==2010].copy()
df2010.to_csv(BASE_DIR.parent.joinpath("Data","22Oct2021","2010_Road_Crashes_SWITRS_TIMS_matched_output_v3.4.csv"))
# df2016 = dfresult[dfresult["ACCIDENT_YEAR"]==2016].copy()
df2016.to_csv(BASE_DIR.parent.joinpath("Data","22Oct2021","2016_Road_Crashes_SWITRS_TIMS_matched_output_v3.4.csv"))

<ipython-input-29-e714b0e29bb8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2010["ref_latlong2"] = df2010["CASE_ID"].map(lstcaseID2010).fillna(df2010["ref_latlong2"])
<ipython-input-29-e714b0e29bb8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2016["ref_latlong2"] = df2016["CASE_ID"].map(lstcaseID2016).fillna(df2016["ref_latlong2"])


In [ ]:
df

In [8]:
df2010 = dfresult[dfresult["ACCIDENT_YEAR"]==2010]
df2016 = dfresult[dfresult["ACCIDENT_YEAR"]==2016]

In [12]:
len(df2010[df2010["ref_latlong2"].isnull()])

605

In [14]:
rd_crash_2010 = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Edited","2010_Road_Crashes_SWITRS_TIMS_matched_output_v3.4.csv"))
lstcaseID2010 = dict(zip(rd_crash_2010[rd_crash_2010["CASE_ID"].isin(dfcaseID2010)]["CASE_ID"],rd_crash_2010[rd_crash_2010["CASE_ID"].isin(dfcaseID2010)]["ref_latlong2"]))

In [19]:
print(next(iter(lstcaseID2010)))

4892462


In [20]:
df2010[df2010["CASE_ID"]==4892462]

,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,...,SECONDARY_RAMP,LATITUDE,LONGITUDE,COUNTY,CITY,POINT_X,POINT_Y,PRIMARY_RD_3,SECONDARY_RD_3,ref_latlong2
2516,4892462,2010,2011-10-12,3801,2010-09-20,2153,2323,NaN,1,5,...,-,NaN,NaN,san francisco,san francisco,NaN,NaN,10TH,SILVER,None


In [24]:
rd_crash_2010.loc[rd_crash_2010["CASE_ID"]==4892462]

,Unnamed: 0,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,...,PRIMARY_RAMP,SECONDARY_RAMP,COUNTY,CITY,PRIMARY_RD_3,SECONDARY_RD_3,ref_latlong2,XY,Unnamed: 83,Unnamed: 84
24,2516,4892462,2010,10/12/2011,3801,9/20/2010,2153,2323,NaN,1,...,-,-,san francisco,san francisco,10TH,SILVER,NaN,NaN,NaN,NaN
